In [0]:
%pip install openai unidecode gspread==5.12.4 tiktoken mlflow
%restart_python
%load_ext autoreload
%autoreload 2 

In [0]:
%restart_python
%load_ext autoreload
%autoreload 2 

In [0]:
from abc import ABC, abstractmethod
import time
import mlflow
from contextlib import contextmanager
import json
import pandas as pd
import datetime
import re
import openai
from openai import OpenAI
import gspread
import random
import logging
import os
import io
from unidecode import unidecode
from pathlib import Path
import pyspark
from pyspark.sql.functions import *
from functools import reduce
from typing import *
import tiktoken


from general_config import *
from ml_tracker import MLTracker
from marketing_config import *
from marketing_localizer import *
#from aso_config import *
#from aso_localizer import *
#from in_game_localizer import InGameLocalizer
#from InGame_Config import * 

In [0]:
%run "./authenticationScript"

In [0]:
#TODO update authenticatinon script and secrets!!
gsheet_client = get_gspread_client_from_secret_old()
gpt_client = get_model_client() 

In [0]:
#PP_ASO_OCTOBER_URL= "https://docs.google.com/spreadsheets/d/1WYl_MiwawlGcgRrXxLNN9WVYgWqqwu393n-9uO7aPhY/edit?gid=1596152423#gid=1596152423"

#DMM_URL = "https://docs.google.com/spreadsheets/d/1r8jIynEvuHz9rz_Gu90dvOj7AcTCSz6A727gTP9DRbY/edit?gid=980819314#gid=980819314"

In [0]:
request =  {"RowFingerprint":"DMM_Marketing",
                    "Timestamp":datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    "SubmitterEmail":"krista@jamcity.com",
                    "DueDate":"9/4/25",
                    'LocType':"Marketing", 
                    'Game':"Disney Magic Match", 
                    'TargetLanguages':"Spanish (Latin America), French, Italian, German, Korean, Japanese, Simplified Chinese, Traditional Chinese (Hong Kong), Portuguese (Brazil)",
                    "URL":"https://docs.google.com/spreadsheets/d/1hL8zavxoRuy1PKToeCbI70RiKHpQUImbRhvcyiLOeFs/edit?gid=0#gid=0",
                    "QAFlag":""}


cfg = {
    "input": 
        {
            "required_tabs": ["input","output"],
        },
    ##add more formatting data for header rows
    "char_limit_policy": "",
    }

In [0]:
#mlflow.active_run()
#mlflow.end_run()

In [0]:
localizer = MarketingLocalizer(request = request, gsheet_client =gsheet_client, gpt_client = gpt_client, cfg=cfg)

In [0]:
localizer.validate_inputs() # GOOD
inputs = localizer.load_inputs()
preprocessed = localizer.preprocess(inputs) #Good
prompts = localizer.build_prompts(preprocessed) #Good


In [0]:
raw_results = localizer.translate(prompts) #Good


In [0]:
postprocessed = localizer.postprocess(raw_results)
localizer.write_outputs(postprocessed)

In [0]:
localizer.tracker.__dict__

In [0]:
#localizer.tracker.run()

In [0]:
#### dbutils.notebook.exit()

In [0]:


number_to_letter = {
            "1": "A",
            "2": "B",
            "3": "C",
            "4": "D",
            "5": "E",
            "6": "F",
            "7": "G",
            "8": "H",
            "9": "I",
            "10": "J",
            "11": "K",
            "12": "L",
            "13": "M",
            "14": "N",
            "15": "O",
            "16": "P",
            "17": "Q",
            "18": "R",
            "19": "S",
            "20": "T",
            "21": "U",
            "22": "V",
            "23": "W",
            "24": "X",
            "25": "Y",
            "26": "Z"
        }

#number_to_letter[str(len(localizer.df.columns.tolist()))]

In [0]:
len(localizer.df.values.tolist())+1

In [0]:
#postprocessed 

In [0]:
results = localizer.results


headers = results.columns.tolist()
values = results.values.tolist()

In [0]:
headers

In [0]:
letter_range = number_to_letter[str(len(headers))]
print(letter_range)

In [0]:
header_range = f"A1:{letter_range}1"
data_range = f"A2:{letter_range}{len(values)+1}"

In [0]:
print(header_range)
print(data_range)

In [0]:
localizer.results.fillna("")

In [0]:
out_sh = localizer.sh.worksheet('output')

out_sh.batch_update([{'range': header_range, 'values': [headers]}])
out_sh.batch_update([{'range': data_range, 'values': values}])

In [0]:
postprocessed = localizer.postprocess(raw_results)
localizer.write_outputs(postprocessed)

In [0]:
dbutils.notebook.exit()